In [115]:
from csv import writer
import datetime 
import random

In [105]:
menu_path = '/content/drive/MyDrive/Colab Notebooks/Menu.txt'
database_path = '/content/drive/MyDrive/Colab Notebooks/Orders_Database.csv'

Projede kaynak olarak sunulan Menu.txt dosyasını dinamik bir yapı oluşturmak için csv kütüphanesi ile okuyoruz.

In [116]:
def get_menu_text():
  with open(menu_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter="\n")
    description = ""
    for i in csv_reader:
      description += i[0]+"\n"
    csv_file.close()
    return description
description = get_menu_text()

Müşterinin verdiği siparişi veri tabanına kaydeden method.

In [117]:
def save_order_to_database(order, total_cost, name, id_no, card_no, card_pass, order_time):
  customer_info_list = [order, total_cost, name, id_no, card_no, card_pass, order_time]

  with open(database_path, 'a') as database:
    writer_object = writer(database)
    writer_object.writerow(customer_info_list)
    database.close()

Okuduğumuz Menu.txt dosyasındaki bilgiler ile
<ul>
<li>ürün <b>numarası-ismi</b>,</li>
<li>ürün <b>ismi-fiyatı</b>(rastgele fiyatlar atadım),</li>
<li>ürün <b>isim-açıklama</b>(bunu kaynak olmadığından kendim yazdım)</li>
</ul>
eşleştirmelerini dictionary kullanarak tanımlıyorum. 

In [118]:
numbers_n_products = {} # pairing numbers and products in a dict
products_n_costs = {}   # defining random prices for products
pizzas_n_descriptions = {
    'Klasik': "Sucuk, Salam, Kaşar Peyniri",
    'Margarita': "Mozarella, Domates, Fesleğen",
    'TürkPizza': "Et Döner, Domates, Soğan",
    'Sade Pizza': "Sucuk, Salam, Domates"
}
liste = description.split("\n") #get a list with rows of description
for i in liste:
  if(i[:2].isdigit()): #check for integer
    numbers_n_products[i[:2]] = i[4:]
    products_n_costs[numbers_n_products[i[:2]]] = random.randint(5,15) #define a random price for the sauce
  else:
    if(i[:1].isdigit()):
      numbers_n_products[i[:1]] = i[3:]
      products_n_costs[numbers_n_products[i[:1]]] = random.randint(50,100) #define a random price for the pizza
print(numbers_n_products)
print(products_n_costs)

{'1': 'Klasik', '2': 'Margarita', '3': 'TürkPizza', '4': 'Sade Pizza', '11': 'Zeytin', '12': 'Mantarlar', '13': 'Keçi Peyniri', '14': 'Et', '15': 'Soğan', '16': 'Mısır'}
{'Klasik': 91, 'Margarita': 67, 'TürkPizza': 96, 'Sade Pizza': 61, 'Zeytin': 5, 'Mantarlar': 9, 'Keçi Peyniri': 10, 'Et': 14, 'Soğan': 6, 'Mısır': 15}




*   Pizza seçimi için bir class, fiyat ve açıklama bilgileri için methodlar tanımlıyoruz.
*  Pizza ve Sosların seçimi için Pizza classından referans alacak Decorator classını tanımlıyoruz.
*  Bir pizza ve isteğe bağlı olarak 1 ila 6 tane arasında sos seçebiliyoruz.
*  Decorator classı, Pizza classından aldığı pizza fiyat ve açıklama bilgisiyle Sosların fiyat ve açıklama bilgisini birleştirip **Toplam Fiyat Bilgisini** ve **Genel Sipariş Açıklamasını** veriyor.





In [119]:
class Pizza:
  def __init__(self, name):
    self.name = name

  def get_cost(self):
    return products_n_costs[self.name]

  def get_description(self):
    return "{} : {}".format(self.name, pizzas_n_descriptions[self.name])

class Decorator(Pizza):
  def __init__(self, name, sos1, sos2=None, sos3=None, sos4=None, sos5=None, sos6=None):
    super().__init__(name)
    self.sos1 = sos1
    self.sos2 = sos2
    self.sos3 = sos3
    self.sos4 = sos4
    self.sos5 = sos5
    self.sos6 = sos6
  def get_total_cost(self):
    sauces_costs = [products_n_costs[j] for j in self.get_sauces_list()]

    return Pizza(self.name).get_cost() + sum(sauces_costs)

  def get_total_description(self):
    sauces_names = ", ".join(self.get_sauces_list())
    return "{} | {}".format(Pizza.get_description(self), sauces_names)

  def get_sauces_list(self):
    lst = [self.sos1, self.sos2, self.sos3, self.sos4, self.sos5, self.sos6]
    sauces_list = [x for x in lst if x!=None] # gets rid of from None values
    selected_sauces_names = [numbers_n_products[i] for i in sauces_list]
    return selected_sauces_names

Sipariş kullanıcıdan alınan verinin sadece menüdeki numara aralığında olmasını sağlayan method. Bu sayede geçersiz inputun önüne geçiyoruz.

In [120]:
def selected_product(name, range_start, range_end):
  numbers_range = [str(i) for i in range(range_start,range_end)]

  sos_list = []

  chosen_product = "something"
  while(chosen_product not in numbers_range):
    chosen_product = input("Seçmek istediğiniz "+name+" numarasını giriniz "+str(numbers_range)[1:-1]+".\n")
    if(chosen_product not in numbers_range):
      print("Geçerli bir değer girin!")
  return chosen_product  

Ana method

In [121]:
def main():
  #Beginning, description
  print("Pizzacı Dükkanına Hoşgeldiniz!\n"+get_menu_text())

  #Store chosen infos
  chosen_pizza_and_sos = [[],[]]
  
  #Get chosen pizza's data
  chosen_pizza_and_sos[0] = selected_product("pizza",1,5)

  #Get chosen sauce's or sauces' data
  chosen_pizza_and_sos[1].append(selected_product("sos",11,17))
  question = "something"
  while(question != "n"):
    question = input("Başka sos seçmek ister misin?(y/n): ")
    if(question == "y"): 
      chosen = selected_product("sos",11,17)
      if(chosen in chosen_pizza_and_sos[1]):
        print("Aynı sosu seçemezsiniz!")
      else:
        chosen_pizza_and_sos[1].append(chosen)
    elif(question == "n"):
      pass
    else:
      print("Geçerli bir değer girin!")
  #---------------------------------

  #print(chosen_pizza_and_sos)

  #Ordering pizza with sauces
  siparis_Pizza_Sos = Decorator(numbers_n_products[chosen_pizza_and_sos[0]], *chosen_pizza_and_sos[1])

  print(siparis_Pizza_Sos.get_total_description())
  print("Toplam ücret :",siparis_Pizza_Sos.get_total_cost(),"TL")


  customer_name = input("İsminiz: ")
  customer_id_no = input("TC Kimlik Numaranız: ")
  customer_card_no = input("Kart Numaranız: ")
  customer_card_pass = input("Kart Şifreniz: ")
  order_time = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
  print("Siparişiniz alınmıştır. 30 dakika içinde teslim alabilirsiniz...")
  save_order_to_database(siparis_Pizza_Sos.get_total_description(), siparis_Pizza_Sos.get_total_cost(), customer_name, customer_id_no, customer_card_no, customer_card_pass, order_time)



In [122]:
main()

Pizzacı Dükkanına Hoşgeldiniz!
* Lütfen Bir Pizza Tabanı Seçiniz:
1: Klasik
2: Margarita
3: TürkPizza
4: Sade Pizza
* ve seçeceğiniz sos:
11: Zeytin
12: Mantarlar
13: Keçi Peyniri
14: Et
15: Soğan
16: Mısır
* Teşekkür ederiz!

Seçmek istediğiniz pizza numarasını giriniz '1', '2', '3', '4'.
2
Seçmek istediğiniz sos numarasını giriniz '11', '12', '13', '14', '15', '16'.
12
Başka sos seçmek ister misin?(y/n): 16
Geçerli bir değer girin!
Başka sos seçmek ister misin?(y/n): y
Seçmek istediğiniz sos numarasını giriniz '11', '12', '13', '14', '15', '16'.
16
Başka sos seçmek ister misin?(y/n): n
Margarita : Mozarella, Domates, Fesleğen | Mantarlar, Mısır
Toplam ücret : 91 TL
İsminiz: Ahmet
TC Kimlik Numaranız: 12345678
Kart Numaranız: 3345566
Kart Şifreniz: 1111
Siparişiniz alınmıştır. 30 dakika içinde teslim alabilirsiniz...
